In [1]:
import os
import json
import pickle
from bisect import bisect_left, bisect_right
from datetime import datetime, timedelta
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
from tqdm import tqdm


In [2]:
DATADIR = "hack_data_lenta" # "./data"
clients = pd.read_csv(f"{DATADIR}/clients.csv")
materials = pd.read_csv(f"{DATADIR}/materials.csv")
plants = pd.read_csv(f"{DATADIR}/plants.csv")

In [3]:
clients

,client_id,gender,city,birthyear
0,1a47d62dddacc03fe90c15652f7ae1a4,M,Other,1990.0
1,9cb909f701d25d548e953bff81192b56,F,Other,1969.0
2,d5da0f5b841b4f83383202807995027a,M,Other,1976.0
3,13ed7f16810b17b8cee6de834ac79a48,F,Moscow,1966.0
4,215fe3ea7d5bf0415e5504e2a7f33551,F,Other,1988.0
...,...,...,...,...
99990,9e7e66fc2c2dcfd8977bc0cffeff8eb5,M,Other,1985.0
99991,e261c97d40ab839926c712b4aa6cefd8,F,St. Petersburg,1982.0
99992,678ef46f41f8c89f7e8578c9e9592e3e,M,Other,1964.0
99993,274facf0609ceba803f5853bd2222208,F,Other,1951.0


In [4]:
materials

,material,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
0,35cbdf61de9e19c8b417327aaef14c88,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,9eed45f71360b4b1e2590637467220e5,212a38db0ddcd009f1e164cc8483485c,0,0
1,c0b0bf24d4ec71da3d304f761ec555d8,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,de2eb747e0896c050905a6b635ab800a,9b0b6c7d55413ad3b67761b7b125b534,0,0
2,f0fc5e654a81a7c4b8ba8d7c26546e14,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,8a969031832c535daf96e0c2aed8e814,9b0b6c7d55413ad3b67761b7b125b534,0,0
3,353693e64fb5f9e2d29746d7fe6edf1e,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0
4,5d9e0b4302ce95448cae72165ff4cf5b,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0
...,...,...,...,...,...,...,...,...
105604,feeddbacd9eed3bac85b1571ac79e40e,NONFOOD,3295c76acbf4caaed33c36b1b5fc2cb1,11704817e347269b7254e744b5e22dac,e47e165040e2914783a451509bbc8180,8ca0d8745f91fb71551be26b1ca854f4,0,0
105605,e218c4c3676927e53b95a12ffe9e721a,FOOD,02e74f10e0327ad868d138f2b4fdd6f0,df0e09d6f25a15a815563df9827f48fa,a03ddfb4b641e98de3223765162217c3,e3319f2f4767af113d82174f662f2d4e,1,1
105606,9a5e2e34d8406616f5981868b4576444,NONFOOD,fc490ca45c00b1249bbe3554a4fdf6fb,865bf46435bd84fa5d89f64cf3ba7347,e2a75db437bee7d34615fccf32d7a464,546614030cdbddcae3de351852962759,0,0
105607,d208910eb82fa1f57684f793f93521a4,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,90365351ccc7437a1309dc64e4db32a3,ea9cffc9bd2bf8a0fb128d223f6a4282,7f1dbcf6ab15c23093508627ca12b8b1,0,0


In [34]:
materials.material.nunique()

105609

In [5]:
plants

,plant,plant_type,city
0,95b09698fda1f64af16708ffb859eab9,HM,St. Petersburg
1,926abae84a4bd33c834bc6b981b8cf30,HM,St. Petersburg
2,ae2bac2e4b4da805d01b2952d7e35ba4,HM,St. Petersburg
3,0e7e3cf0ded4d9db8b376b317c007f99,HM,St. Petersburg
4,540bd55a2cf295b8ea9cd78650e89d03,HM,St. Petersburg
...,...,...,...
383,8178a650c360ce1a3fe81858f0e75760,SM,Other
384,238d77348d954fdd455aabb09575f2d0,SM,Other
385,6e1acaeb838a890e6570284f73e1d00f,SM,Other
386,7fc3ee9aad33036cc501b2ca7c163fc1,SM,Other


In [35]:
plants.city.value_counts()

Other             316
St. Petersburg     46
Moscow             26
Name: city, dtype: int64

In [12]:
transactions = pd.read_parquet(f"{DATADIR}/transactions.parquet")

In [37]:
transactions['date'] 

(32109414, 10)

In [38]:
transactions['year'] = transactions['date'].dt.year 
transactions['month'] = transactions['date'].dt.month 

In [39]:
y = [2016, 2017]
m = range(1, 13)
for yi in y:
    for mi in m:
        c1 = transactions['year'] == yi
        c2 = transactions['month'] == mi
        t_new = transactions[c1&c2]
        t_new.to_csv(f"trans_{yi}_{mi}")

In [21]:
transactions['date'] = pd.to_datetime(transactions.chq_date)

In [71]:
popular_mat = transactions.material.value_counts()[transactions.material.value_counts() > 1000].index

In [72]:
pop_mat_ind = [i in popular_mat for i in transactions.material]

In [68]:
transactions

,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo,date,year,month
0,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,004,90fca68123e692a005a4edeadd94b2f2,232cfe8b28970434f685b737737b26d4,2.000,146.98,0,2016-11-01,2016,11
1,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,003,90fca68123e692a005a4edeadd94b2f2,12ec64451b8e426db16f3f05ea770901,1.000,249.99,1,2016-11-01,2016,11
2,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,012,90fca68123e692a005a4edeadd94b2f2,1212ff8ecf53aebef093ca8ae216db68,1.000,47.99,1,2016-11-01,2016,11
3,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,013,90fca68123e692a005a4edeadd94b2f2,1212ff8ecf53aebef093ca8ae216db68,1.000,47.99,1,2016-11-01,2016,11
4,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,011,90fca68123e692a005a4edeadd94b2f2,571c6d3b559db8445e05265649eb30ab,2.000,53.98,1,2016-11-01,2016,11
...,...,...,...,...,...,...,...,...,...,...,...,...
32109409,37b3f0cdc086f820942fdcad8f8b583e,6d9ff949640422493f3db836c3035c64,2016-10-31,003,4cc84331fe90f77681ad374faaf86317,82cfb8081737e152146b7bc10441bd00,0.795,215.44,1,2016-10-31,2016,10
32109410,b1a59187a28741b3cc8c7e2a51711e8a,6d9ff949640422493f3db836c3035c64,2016-10-31,001,7ed222a4d1772e52669d844ec9830a9a,55ad93c2949b2ac7b3c0fe4dae8ead3d,1.840,34.94,1,2016-10-31,2016,10
32109411,b1a59187a28741b3cc8c7e2a51711e8a,6d9ff949640422493f3db836c3035c64,2016-10-31,003,7ed222a4d1772e52669d844ec9830a9a,55ad93c2949b2ac7b3c0fe4dae8ead3d,1.710,32.47,1,2016-10-31,2016,10
32109412,b1a59187a28741b3cc8c7e2a51711e8a,6d9ff949640422493f3db836c3035c64,2016-10-31,004,7ed222a4d1772e52669d844ec9830a9a,96dc1b235835fe9a29d0e9122a17ace3,1.000,21.10,0,2016-10-31,2016,10


In [80]:
gb = transactions[pop_mat_ind].groupby(['plant', 'material', 'date'])[['sales_count', 'is_promo']].sum()

In [67]:
materials[materials['material'] == '000062284ffb915ba6b88a36e3cb7ba8']

,material,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
48001,000062284ffb915ba6b88a36e3cb7ba8,FOOD,70efdf2ec9b086079795c442636b55fb,a588a6199feff5ba48402883d9b72700,d390d3da2959fcf07b08b99257b0f310,f605bc9f437fd0a6569e6143bde52ee9,0,0


In [76]:
popular_mat[5]

'ab17bf24039893348f6b9f23753f38ca'

In [81]:
gb1 = gb.reset_index()

In [86]:
gb1[gb1['material'] == popular_mat[7]]

,plant,material,date,sales_count,is_promo
14401,00fdea78e677259a3e44868468f24bbf,5a008f40c57611feda53dbd9935e7189,2016-10-09,0.560,1
14402,00fdea78e677259a3e44868468f24bbf,5a008f40c57611feda53dbd9935e7189,2016-10-11,0.420,1
14403,00fdea78e677259a3e44868468f24bbf,5a008f40c57611feda53dbd9935e7189,2016-10-12,0.502,0
14404,00fdea78e677259a3e44868468f24bbf,5a008f40c57611feda53dbd9935e7189,2016-10-13,1.574,0
14405,00fdea78e677259a3e44868468f24bbf,5a008f40c57611feda53dbd9935e7189,2016-10-16,0.430,1
...,...,...,...,...,...
16982382,ffe49cdf45d56b04b8542e6abc4f5fb6,5a008f40c57611feda53dbd9935e7189,2017-09-12,1.396,0
16982383,ffe49cdf45d56b04b8542e6abc4f5fb6,5a008f40c57611feda53dbd9935e7189,2017-09-27,1.614,0
16982384,ffe49cdf45d56b04b8542e6abc4f5fb6,5a008f40c57611feda53dbd9935e7189,2017-09-29,0.338,0
16982385,ffe49cdf45d56b04b8542e6abc4f5fb6,5a008f40c57611feda53dbd9935e7189,2017-10-01,1.038,0


In [83]:
gb1[gb1['plant']=='00fdea78e677259a3e44868468f24bbf'][gb1['material'] == popular_mat[5]].head(20)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,plant,material,date,sales_count,is_promo
28283,00fdea78e677259a3e44868468f24bbf,ab17bf24039893348f6b9f23753f38ca,2016-11-17,0.276,0
28284,00fdea78e677259a3e44868468f24bbf,ab17bf24039893348f6b9f23753f38ca,2016-11-18,1.120,0
28285,00fdea78e677259a3e44868468f24bbf,ab17bf24039893348f6b9f23753f38ca,2016-11-19,0.890,1
28286,00fdea78e677259a3e44868468f24bbf,ab17bf24039893348f6b9f23753f38ca,2016-11-21,2.388,0
28287,00fdea78e677259a3e44868468f24bbf,ab17bf24039893348f6b9f23753f38ca,2016-11-24,0.954,0
28288,00fdea78e677259a3e44868468f24bbf,ab17bf24039893348f6b9f23753f38ca,2016-11-26,0.740,0
28289,00fdea78e677259a3e44868468f24bbf,ab17bf24039893348f6b9f23753f38ca,2016-11-27,3.362,0
28290,00fdea78e677259a3e44868468f24bbf,ab17bf24039893348f6b9f23753f38ca,2016-11-28,0.414,0
28291,00fdea78e677259a3e44868468f24bbf,ab17bf24039893348f6b9f23753f38ca,2016-12-03,1.420,0
28292,00fdea78e677259a3e44868468f24bbf,ab17bf24039893348f6b9f23753f38ca,2016-12-06,1.642,1


In [ ]:
#Data preprocess

In [100]:
uniq_chq = transactions.chq_id.unique()
dict_uniq_chq = dict(zip(uniq_chq,range(len(uniq_chq))))
def dict_chq(x):
    return dict_uniq_chq[x]
transactions['chq_id_cat'] = transactions.chq_id.apply(dict_chq)

In [102]:
uniq_mat = transactions.material.unique()
dict_uniq_mat = dict(zip(uniq_mat,range(len(uniq_mat))))
def dict_mat(x):
    return dict_uniq_mat[x]
transactions['material_cat'] = transactions.material.apply(dict_mat)

In [103]:
uniq_clt = transactions.client_id.unique()
dict_uniq_clt = dict(zip(uniq_clt,range(len(uniq_clt))))
def dict_clt(x):
    return dict_uniq_clt[x]
transactions['client_id_cat'] = transactions.client_id.apply(dict_clt)

In [107]:
uniq_plt = transactions.plant.unique()
dict_uniq_plt = dict(zip(uniq_plt,range(len(uniq_plt))))
def dict_plt(x):
    return dict_uniq_plt[x]
transactions['plant_cat'] = transactions.plant.apply(dict_plt)

In [108]:
data = transactions.drop(['chq_id', 'plant', 'chq_date', 'client_id', 'material'], axis=1)

In [109]:
data

,chq_position,sales_count,sales_sum,is_promo,date,year,month,chq_id_cat,material_cat,client_id_cat,plant_cat
0,004,2.000,146.98,0,2016-11-01,2016,11,0,0,0,0
1,003,1.000,249.99,1,2016-11-01,2016,11,0,1,0,0
2,012,1.000,47.99,1,2016-11-01,2016,11,0,2,0,0
3,013,1.000,47.99,1,2016-11-01,2016,11,0,2,0,0
4,011,2.000,53.98,1,2016-11-01,2016,11,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...
32109409,003,0.795,215.44,1,2016-10-31,2016,10,2964773,43155,2913,187
32109410,001,1.840,34.94,1,2016-10-31,2016,10,2964774,680,64170,187
32109411,003,1.710,32.47,1,2016-10-31,2016,10,2964774,680,64170,187
32109412,004,1.000,21.10,0,2016-10-31,2016,10,2964774,35138,64170,187


In [111]:
Npoppular_mat = 1000
popular_mat = data.material_cat.value_counts()[transactions.material_cat.value_counts() > Npoppular_mat].index
pop_mat_ind = [i in popular_mat for i in data.material_cat]
trans_pop_mat = data[pop_mat_ind]
gb1 = trans_pop_mat.groupby(['plant_cat', 'material_cat', 'date'])[['sales_count', 'sales_sum']].sum()
gb2 = trans_pop_mat.groupby(['plant_cat', 'material_cat', 'date'])['chq_id_cat'].count()
gb1 = gb1.reset_index()
gb2 = gb2.reset_index().rename(columns={'chq_id_cat': 'chq_id_cat_cnt'})
gb = gb1.merge(gb2, on=['plant_cat', 'material_cat', 'date'], how='left')

In [121]:
product_dict = {'Not sold' : 0, 'less frequently' :1, 'usual' :2, 'more frequently' :3}

In [123]:
gb['year'] = gb['date'].dt.year
gb['month'] = gb['date'].dt.month

In [125]:
mean_sales = gb.groupby(['plant_cat', 'material_cat', 'year', 'month'])['sales_count'].mean()
std_sales = gb.groupby(['plant_cat', 'material_cat', 'year', 'month'])['sales_count'].std()
mean_cnt = gb.groupby(['plant_cat', 'material_cat', 'year', 'month'])['chq_id_cat_cnt'].mean()

In [127]:
mean_sales = mean_sales.reset_index().rename(columns={'sales_count': 'mean_sales'})
gb = gb.merge(mean_sales, on=['plant_cat', 'material_cat', 'year', 'month'], how='left')
mean_cnt = mean_cnt.reset_index().rename(columns={'chq_id_cat_cnt': 'mean_cnt'})
gb = gb.merge(mean_cnt, on=['plant_cat', 'material_cat', 'year', 'month'], how='left')
std_sales = std_sales.reset_index().rename(columns={'sales_count': 'std_sales'})
gb = gb.merge(std_sales, on=['plant_cat', 'material_cat', 'year', 'month'], how='left')

In [130]:
gb['std_sales'] = gb['std_sales'].fillna(0).astype(float)

In [140]:
gb.loc[0]

plant_cat                           0
material_cat                        0
date              2016-10-04 00:00:00
sales_count                         3
sales_sum                      219.54
chq_id_cat_cnt                      1
year                             2016
month                              10
mean_sales                    3.41176
mean_cnt                      1.94118
std_sales                     2.18114
Name: 0, dtype: object

In [141]:
import math

In [144]:
gb['mat_freq'] = [0]*gb.shape[0]

In [145]:
gb

,plant_cat,material_cat,date,sales_count,sales_sum,chq_id_cat_cnt,year,month,mean_sales,mean_cnt,std_sales,mat_freq
0,0,0,2016-10-04,3.0,219.54,1,2016,10,3.411765,1.941176,2.181136,0
1,0,0,2016-10-05,1.0,73.49,1,2016,10,3.411765,1.941176,2.181136,0
2,0,0,2016-10-07,3.0,220.47,2,2016,10,3.411765,1.941176,2.181136,0
3,0,0,2016-10-08,7.0,514.42,2,2016,10,3.411765,1.941176,2.181136,0
4,0,0,2016-10-11,1.0,73.49,1,2016,10,3.411765,1.941176,2.181136,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16986882,387,18446,2016-10-04,1.0,199.01,1,2016,10,1.000000,1.000000,0.000000,0
16986883,387,18802,2016-10-07,2.0,197.18,2,2016,10,2.000000,2.000000,0.000000,0
16986884,387,19369,2016-10-06,1.0,49.99,1,2016,10,1.000000,1.000000,0.000000,0
16986885,387,19830,2016-10-04,2.0,65.98,1,2016,10,2.000000,1.000000,0.000000,0


In [156]:
for idx in gb.index:
    if (gb.loc[idx]['sales_count'] < gb.loc[idx]['mean_sales']-gb.loc[idx]['std_sales'] and gb.loc[idx]['chq_id_cat_cnt'] < math.floor(0.8*gb.loc[idx]['mean_cnt'])):
        gb.loc[idx,'mat_freq'] = product_dict['less frequently']
    elif (gb.loc[idx]['sales_count'] > gb.loc[idx]['mean_sales']+gb.loc[idx]['std_sales']):
        gb.loc[idx,'mat_freq'] = product_dict['more frequently']
    else:
        gb.loc[idx,'mat_freq'] = product_dict['usual']

KeyboardInterrupt: 

In [158]:
gb[gb.mat_freq != 0]

,plant_cat,material_cat,date,sales_count,sales_sum,chq_id_cat_cnt,year,month,mean_sales,mean_cnt,std_sales,mat_freq
0,0,0,2016-10-04,3.0,219.54,1,2016,10,3.411765,1.941176,2.181136,2
1,0,0,2016-10-05,1.0,73.49,1,2016,10,3.411765,1.941176,2.181136,2
2,0,0,2016-10-07,3.0,220.47,2,2016,10,3.411765,1.941176,2.181136,2
3,0,0,2016-10-08,7.0,514.42,2,2016,10,3.411765,1.941176,2.181136,3
4,0,0,2016-10-11,1.0,73.49,1,2016,10,3.411765,1.941176,2.181136,2
...,...,...,...,...,...,...,...,...,...,...,...,...
28689,0,1237,2017-05-14,1.0,14.79,1,2017,5,2.250000,1.000000,2.500000,2
28690,0,1237,2017-06-21,2.0,26.68,1,2017,6,10.000000,1.500000,11.313708,2
28691,0,1237,2017-06-25,18.0,377.00,2,2017,6,10.000000,1.500000,11.313708,2
28692,0,1237,2017-07-05,1.0,13.39,1,2017,7,1.142857,1.000000,0.377964,2


In [155]:
gb.loc[0,'mat_freq'] = product_dict['usual']

In [154]:
gb.loc[0,'mat_freq'] = 2

In [120]:
for plant, material in zip(uniq_plt, material_cat):
    gb[''

,plant_cat,material_cat,date,sales_count,sales_sum,chq_id_cat_cnt
0,0,0,2016-10-04,3.0,219.54,1
1,0,0,2016-10-05,1.0,73.49,1
2,0,0,2016-10-07,3.0,220.47,2
3,0,0,2016-10-08,7.0,514.42,2
4,0,0,2016-10-11,1.0,73.49,1
...,...,...,...,...,...,...
16986882,387,18446,2016-10-04,1.0,199.01,1
16986883,387,18802,2016-10-07,2.0,197.18,2
16986884,387,19369,2016-10-06,1.0,49.99,1
16986885,387,19830,2016-10-04,2.0,65.98,1


In [ ]:
gp2 = gb2.reset_index().rename(columns={'price': 'price_by_yr_month_std'})

In [88]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32109414 entries, 0 to 32109413
Data columns (total 12 columns):
chq_id          object
plant           object
chq_date        datetime64[ns]
chq_position    object
client_id       object
material        object
sales_count     float64
sales_sum       float64
is_promo        int64
date            datetime64[ns]
year            int64
month           int64
dtypes: datetime64[ns](2), float64(2), int64(3), object(5)
memory usage: 2.9+ GB


In [16]:
transactions.is_promo

0    16588243
1    15521171
Name: is_promo, dtype: int64

In [15]:
transactions[transactions.sales_sum < 0].is_promo.value_counts()

0    13786
1     8987
Name: is_promo, dtype: int64